## Predicting High Priority Tickets: To predict priority 1 & 2 tickets, so that they can take preventive measures or fix the problem before it surfaces.

## Insights drawn:

In [11]:
from sqlalchemy import create_engine
import pandas as pd

In [12]:
db_host = '18.136.56.185'
username ='dm_team'
pwd='dm_team123#'
db_name='project_itsm'

In [13]:
import mysql.connector

db = mysql.connector.connect(host=db_host,
                             user=username,
                             password=pwd
                             )

cursor =db.cursor()

cursor.execute("SHOW DATABASES")

database =cursor.fetchall()

print(database)

[('information_schema',), ('project_itsm',)]


In [14]:
#Connecting to SQL DB
conn =create_engine('mysql+pymysql://'+username+':'+pwd+'@'+db_host+'/'+db_name)
conn.table_names()

['dataset_list']

### Load Data

In [15]:
query ='select * from dataset_list '
df =pd.read_sql(query,conn)
print(df.shape)
df.head()



(46606, 25)


,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4,0.601292279,...,,04-11-2013 13:50,04-11-2013 13:51,"3,87,16,91,111",Other,1,SD0000007,2,,
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3,0.415049969,...,02-12-2013 12:31,02-12-2013 12:36,02-12-2013 12:36,"4,35,47,86,389",Software,1,SD0000011,1,,
2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NA,0.517551335,...,,13-01-2014 15:12,13-01-2014 15:13,"4,84,31,19,444",No error - works as designed,1,SD0000017,,,
3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4,0.642927218,...,,14-11-2013 09:31,14-11-2013 09:31,"4,32,18,33,333",Operator error,1,SD0000025,,,
4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4,0.345258343,...,,08-11-2013 13:55,08-11-2013 13:55,"3,38,39,03,333",Other,1,SD0000029,,,


In [ ]:
df.columns
df.dtypes


## EDA

In [16]:
from collections import Counter
import numpy as np

# CI_Cat filed cleaning 
Counter(df.CI_Cat)
df.CI_Cat.isnull().sum()

#Replacing empty srting values of 111 records of CI_Cat column n dropping the same

df['CI_Cat'] = np.where((df.CI_Cat == ''),np.nan,df.CI_Cat)
Counter(df['CI_Cat'])

df.dropna(inplace=True)

df.isnull().sum()
Counter(df['CI_Cat'])


Counter({'subapplication': 7782,
         'application': 32900,
         'computer': 3643,
         'displaydevice': 212,
         'software': 333,
         'storage': 703,
         'database': 214,
         'hardware': 442,
         'officeelectronics': 152,
         'networkcomponents': 107,
         'applicationcomponent': 5,
         'Phone': 2})

### Cleaning columns : Impact,Urgency,Priority

In [17]:
#-----------Impact column have junk values 'NS' so it has to be replced by valid value
df.loc[(df.Impact=='NS'),'Impact']= df.Urgency
Counter(df.Impact)

#-----------Urgency column have '5 - Very Low': rename it to 5
df.loc[(df.Urgency=='5 - Very Low')] = '5'

#----------Priority column has NA values replce with urgency values
Counter(df.Priority)
df.loc[(df.Priority == 'NA'),'Priority'] =df.Urgency
df.loc[(df.Priority==5),'Priority'] ='5'

Counter(df.Priority)

Counter({'4': 22721, '3': 6655, '5': 16427, '2': 689, '1': 3})

### Cleaning WBS column

In [18]:
df.columns
Counter(df.WBS)

df.loc[(df.WBS == '5'),'WBS'] = np.nan


df.isnull().sum()
df.loc[:,'WBS'] = df.loc[:,'WBS'].ffill()
#df.WBS = df['WBS'].ffill()
df.isnull().sum()

Counter(df.WBS)


Counter({'WBS000162': 495,
         'WBS000088': 619,
         'WBS000092': 784,
         'WBS000055': 115,
         'WBS000090': 128,
         'WBS000073': 13342,
         'WBS000066': 154,
         'WBS000071': 21,
         'WBS000263': 2283,
         'WBS000072': 2197,
         'WBS000054': 165,
         'WBS000271': 1186,
         'WBS000170': 70,
         'WBS000014': 219,
         'WBS000098': 208,
         'WBS000043': 232,
         'WBS000086': 80,
         'WBS000027': 166,
         'WBS000300': 43,
         'WBS000223': 803,
         'WBS000311': 16,
         'WBS000142': 2,
         'WBS000109': 1,
         'WBS000153': 146,
         'WBS000102': 185,
         'WBS000152': 1076,
         'WBS000129': 2,
         'WBS000251': 42,
         'WBS000115': 17,
         'WBS000168': 72,
         'WBS000136': 18,
         'WBS000314': 3,
         'WBS000187': 353,
         'WBS000135': 54,
         'WBS000217': 70,
         'WBS000091': 2500,
         'WBS000296': 582,
         'WBS

### Label ENcoding of columns CI_Cat ,CI_SubCat and Catogery


In [19]:
from sklearn.preprocessing import LabelEncoder

encoder= LabelEncoder()

df.CI_Cat = encoder.fit_transform(df['CI_Cat'])
df.CI_Cat 

df.CI_Subcat =encoder.fit_transform(df['CI_Subcat'])
df.CI_Subcat

Counter(df.Category)
df.Category = encoder.fit_transform(df.Category)
df.Category

df.WBS = encoder.fit_transform(df.WBS)
df.WBS

0        136
1         70
2         74
3         70
4         70
        ... 
46601     59
46602     59
46603     73
46604     59
46605    216
Name: WBS, Length: 46495, dtype: int32

### Define X & Y   , split the data

In [75]:
#Since Priority can be determined by considering the Impact and Urgency
X=df.loc[:,['CI_Cat','CI_Subcat','Category','WBS','Impact']]
Y=df.Priority


In [76]:
from sklearn.model_selection import train_test_split

#Dividing data into train n test dataset
X_train ,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=10)


### PCA for detremine the significancec of each columns

In [77]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [78]:
from sklearn.decomposition import PCA
pca =PCA()
X1=pd.DataFrame(pca.fit_transform(X_std))

In [79]:
print(X.columns)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_)

Index(['CI_Cat', 'CI_Subcat', 'Category', 'WBS', 'Impact'], dtype='object')
[0.3093569  0.26706784 0.16990137 0.15074636 0.10292753]
[1.54681779 1.3353679  0.8495251  0.75374804 0.51464872]


### Define Model :  RandomForestClassifier

In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score

model_rf = RandomForestClassifier(max_depth= 15, n_estimators= 1000)
model_rf.fit(X_train,Y_train)
Y_predict= model_rf.predict(X_test)

accuracy_score(Y_test,Y_predict)

0.9952684780270987

In [66]:
print(confusion_matrix(Y_test,Y_predict))


[[   0    0    0    1    0]
 [   0  130   17   45   13]
 [   0   71 1188  509  157]
 [   0   62  215 6219  327]
 [   0   19   42  865 4069]]


In [67]:
from sklearn.svm import SVC

model_bernb =SVC(C=2,gamma=1 ,kernel='rbf' ,probability=True)

model_bernb.fit(X_train,Y_train)
Y_predict = model_bernb.predict(X_test)

accuracy_score(Y_test,Y_predict)

0.831815900781418

### HyperParameter tuning using GridSearchCV

In [34]:
from sklearn.model_selection import GridSearchCV

params={'n_estimators' : [5,10,100,500,250],
       'max_depth' : [2,4,8,10,15]}

cv =GridSearchCV(model_rf ,params ,cv=5)
cv.fit(X_train,Y_train)


C:\Users\Rakshu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=8,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=1000, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='war

In [36]:
print("Best Estimators :  ", cv.best_estimator_)
print("Best Score : " , cv.best_score_)
print("Best Params : ",cv.best_params_)

Best Estimators :   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=15, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Best Score :  0.8209611012105943
Best Params :  {'max_depth': 15, 'n_estimators': 500}


## Model2

In [81]:
X1=df.loc[:,['CI_Cat','CI_Subcat','Category','WBS']]
Y1=df.Impact

X_train1 ,X_test1,Y_train1,Y_test1 = train_test_split(X1,Y1,test_size=0.3,random_state=10)

model_rf1 = RandomForestClassifier(max_depth= 15, n_estimators= 1000)
model_rf1.fit(X_train1,Y_train1)
Y_predict1= model_rf1.predict(X_test1)

accuracy_score(Y_test1,Y_predict1)

0.8330346261380744

In [86]:
X1=df.loc[:,['CI_Cat','CI_Subcat','Category','WBS']]
Y1=df.Urgency

model_rf2 = RandomForestClassifier(max_depth= 15, n_estimators= 1000)
model_rf2.fit(X_train1,Y_train1)
Y_predict2= model_rf2.predict(X_test1)

accuracy_score(Y_test1,Y_predict2)

0.8278729658040003